In [1]:
import numpy as np
import time
import sys

from matplotlib import pyplot as plt
from scipy.interpolate import interp1d

from classy import Class

In [2]:
# First make sure we can generate halofit power spectra from Class
def chi_ls(OmegaM, h):

    lnAs = 2.8
    
    omega_b = 0.02242
    ns = 0.9665

    nnu = 1
    nur = 2.033
    mnu = 0.06
    omega_nu = 0.0106 * mnu
        
    omega_c = (OmegaM - omega_b/h**2 - omega_nu/h**2) * h**2

    pkparams = {
            'A_s': np.exp(lnAs)*1e-10,
            'n_s': ns,
            'h': h,
            'N_ur': nur,
            'N_ncdm': nnu,
            'm_ncdm': mnu,
            'tau_reio': 0.0568,
            'omega_b': omega_b,
            'omega_cdm': omega_c}

    #t1 = time.time()
    pkclass = Class()
    pkclass.set(pkparams)
    pkclass.compute()
    #t2 = time.time()
    #print(t2-t1)
    
    thermo = pkclass.get_thermodynamics()
    bg = pkclass.get_background()
    
    chi_func = interp1d(bg['z'],bg['comov. dist.'],kind='cubic')
    
    return chi_func(thermo['z'][np.argmax(thermo['g [Mpc^-1]'])]) * h

In [3]:
chi_ls(0.31,0.68)

9407.213752114458

In [4]:
# Load data per z:
# These are settings for OmegaM, h, lnAs
param_str = ['omegam', 'h']
x0s = [0.31, 0.68]
dxs = [0.01, 0.01]

order = 3
Nparams = len(x0s)

template = np.arange(-order,order+1,1)
Npoints = 2*order + 1
grid_axes = [ dx*template + x0 for x0, dx in zip(x0s,dxs)]

In [5]:
grid_axes

[array([0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34]),
 array([0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71])]

In [6]:
center_ii = (order,)*Nparams
chigrid = np.zeros( (Npoints,)*Nparams)

for ii in range(Npoints):
    for jj in range(Npoints):
        #print(ii,jj)
        OmegaM, h = grid_axes[0][ii], grid_axes[1][jj]
        chigrid[ii,jj] = chi_ls(OmegaM, h)

In [7]:
# Now compute the derivatives
from taylor_approximation import compute_derivatives, taylor_approximate
derivs = compute_derivatives(chigrid, dxs, center_ii, 5)

In [8]:
# Try Emulating!
test_point = [0.35, 0.75]

import time
t1 = time.time()
chitest = taylor_approximate(test_point, x0s, derivs, order=3)
t2 = time.time()
print(t2-t1)

0.0008513927459716797


In [9]:
print(chitest, chitest/chi_ls(*test_point)-1)

8965.04766293233 -0.0003369296414293954


In [10]:
# Save a json file
import json
import os

basedir = os.getcwd()

listo = [ dd.tolist() for dd in derivs ]
# and add them to the dictionary.
emu_dict = {'params': param_str,\
            'x0': x0s,\
            'derivs': listo}

# Write the results to file.
outfile   = basedir + '/emu/chi_ls.json'
json_file = open(outfile, 'w')
json.dump(emu_dict, json_file)
json_file.close()
#

In [11]:
# test emulator
from emulator_chils import Chi_LS

chifunc = Chi_LS(outfile)
chifunc(test_point)

8965.04766293233

In [15]:
chifunc(test_point)/chi_ls(*test_point)-1

-0.0003369296414293954